<a href="https://colab.research.google.com/github/nagavamsik/Vamsikrishna_labs_projects/blob/main/FMML_M1L3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Augmentation

FMML Module 1, Lab 3

In this lab, we will see how augmentation of data samples help in improving the machine learning performance. Augmentation is the process of creating new data samples by making reasonable modifications to the original data samples. This is particularly useful when the size of the training data is small. We will use the MNISt dataset for this lab. We will also reuse functions from the previous labs.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn.utils.extmath import cartesian
from skimage.transform import rotate, AffineTransform, warp

rng = np.random.default_rng(seed=42)

In [ ]:
# loading the dataset
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# normalizing the data
train_X = train_X / 255
test_X = test_X / 255

# subsample from images and labels. Otherwise it will take too long!
train_X = train_X[::1200, :, :].copy()
train_y = train_y[::1200].copy()

Let us borrow a few functions from the previous labs:

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    traindata = traindata.reshape(-1, 28*28)
    testdata = testdata.reshape(-1, 28*28)
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel


def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

In this lab, we will use the image pixels themselves as features, instead of extracting features. Each image has 28*28 pixels, so we will flatten them to 784 pixels to use as features. Note that this is very compute intensive and will take a long time. Let us first check the baseline accuracy on the test set without any augmentations. We hope that adding augmentations will help us to get better results.

In [ ]:
testpred = NN(train_X, train_y, test_X)
print("Baseline accuracy without augmentation:",
      Accuracy(test_y, testpred)*100, "%")

Let us try to improve this accuracy using augmentations. When we create augmentations, we have to make sure that the changes reflect what will naturally occur in the dataset. For example, we should not add colour to our samples as an augmentation because they do not naturally occur. We should not also flip the images in MNIST, because flipped images have different meanings for digits. So, we will use the following augmentations:

### Augmentation 1: Rotation

Let us try rotating the image a little. We will use the `rotate` function from the `skimage` module. We will rotate the image by 10 degrees and -10 degrees. Rotation is a reasonable augmentation because the digit will still be recognizable even after rotation and is representative of the dataset.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

axs[0].imshow(train_X[2], cmap="gray")
axs[0].set_title("Original Image")

axs[1].imshow(rotate(train_X[2], 10), cmap="gray")
axs[1].set_title("Rotate +10 degrees")

axs[2].imshow(rotate(train_X[2], -10), cmap="gray")
axs[2].set_title("Rotate -10 degrees")

plt.show()

After rotating, the the class of the image is still the same. Let us make a function to rotate multiple images by random angles. We want a slightly different image every time we run this function. So, we generate a random number between 0 and 1 and change it so that it lies between -constraint/2 and +constraint/2

In [ ]:
def augRotate(sample, angleconstraint):
    """
    This function takes in a sample and an angle constraint and returns the augmented sample
    by rotating the sample by a random angle within the angle constraint

    sample: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    angleconstraint: the maximum angle by which the sample can be rotated

    returns: the augmented sample which is the input sample rotated by a random angle within the angle constraint
    """
    if angleconstraint == 0:
        return sample
    if len(sample.shape) == 2:
        # make sure the sample is 3 dimensional
        sample = np.expand_dims(sample, 0)
    angle = rng.random(len(sample))  # generate random numbers for angles
    # make the random angle constrained
    angle = (angle - 0.5) * angleconstraint
    nsample = sample.copy()  # preallocate the augmented array to make it faster
    for ii in range(len(sample)):
        nsample[ii] = rotate(sample[ii], angle[ii])
    return np.squeeze(nsample)  # take care if the input had only one sample.

This function returns a slightly different image each time we call it. So we can increase the number of images in the sample by any multiple.

In [ ]:
sample = train_X[20]
angleconstraint = 70

fig, axs = plt.subplots(1, 5, figsize=(15, 5))

axs[0].imshow(sample, cmap="gray")
axs[0].set_title("Original Image")

axs[1].imshow(augRotate(sample, angleconstraint), cmap="gray")
axs[1].set_title("Aug. Sample 1")

axs[2].imshow(augRotate(sample, angleconstraint), cmap="gray")
axs[2].set_title("Aug. Sample 2")

axs[3].imshow(augRotate(sample, angleconstraint), cmap="gray")
axs[3].set_title("Aug. Sample 3")

axs[4].imshow(augRotate(sample, angleconstraint), cmap="gray")
axs[4].set_title("Aug. Sample 4")

plt.show()

Let us augment the whole dataset and see if this improves the test accuracy

In [ ]:
# hyperparameters
angleconstraint = 60
naugmentations = 5

# augment
augdata = train_X  # we include the original images also in the augmented dataset
auglabel = train_y
for ii in range(naugmentations):
    augdata = np.concatenate(
        (augdata, augRotate(train_X, angleconstraint))
    )  # concatenate the augmented data to the set
    auglabel = np.concatenate(
        (auglabel, train_y)
    )  # the labels don't change when we augment

# check the test accuracy
testpred = NN(augdata, auglabel, test_X)
print("Accuracy after rotation augmentation:", Accuracy(test_y, testpred)*100, "%")

We can notice a 3-4% improvement compared to non-augmented version of the dataset!

The angle constraint is a hyperparameter which we have to tune using a validation set. (Here we are not doing that for time constraints). Let us try a grid search to find the best angle constraint. We will try angles between 0 and 90 degrees. We can also try different multiples of the original dataset. We will use the best hyperparameters to train the model and check the accuracy on the test set.

In [ ]:
angleconstraints = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]  # the values we want to test
accuracies = np.zeros(
    len(angleconstraints), dtype=float
)  # we will save the values here

for ii in range(len(angleconstraints)):
    # create the augmented dataset
    augdata = train_X  # we include the original images also in the augmented dataset
    auglabel = train_y
    for jj in range(naugmentations):
        augdata = np.concatenate(
            (augdata, augRotate(train_X, angleconstraints[ii]))
        )  # concatenate the augmented data to the set
        auglabel = np.concatenate(
            (auglabel, train_y)
        )  # the labels don't change when we augment

    # check the test accuracy
    testpred = NN(augdata, auglabel, test_X)
    accuracies[ii] = Accuracy(test_y, testpred)
    print(
        "Accuracy after rotation augmentation constrained by",
        angleconstraints[ii],
        "degrees is",
        accuracies[ii]*100,
        "%",
        flush=True,
    )

Let us see the best value for angle constraint: (Ideally this should be done on validation set, not test set)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
# plot the variation of accuracy
ax.plot(angleconstraints, accuracies)
ax.set_xlabel("angle")
ax.set_ylabel("accuracy")

# plot the maximum accuracy
maxind = np.argmax(accuracies)
plt.scatter(angleconstraints[maxind], accuracies[maxind], c="red")

### Augmentation 2: Shear


Let us try one more augmentation: shear. Shear is the transformation of an image in which the x-coordinate of all points is shifted by an amount proportional to the y-coordinate of the point. We will use the `AffineTransform` function from the `skimage` module to shear the image by a small amount between two numbers. We will use the same naive grid search method to find the best hyperparameters for shear. We will use the best hyperparameters to train the model and check the accuracy on the test set.

In [ ]:
def shear(sample, amount):
    """
    This function takes in a sample and an amount and returns the augmented sample
    by shearing the sample by the given amount

    sample: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    amount: the amount by which the sample should be sheared

    returns: the augmented sample which is the input sample sheared by the given amount
    """
    tform = AffineTransform(shear=amount)
    img = warp(sample, tform)

    # Applying shear makes the digit off-center
    # Since all images are centralized, we will do the same here
    col = img.sum(0).nonzero()[0]
    row = img.sum(1).nonzero()[0]
    if len(col) > 0 and len(row) > 0:
        xshift = int(sample.shape[0] / 2 - (row[0] + row[-1]) / 2)
        yshift = int(sample.shape[1] / 2 - (col[0] + col[-1]) / 2)
        img = np.roll(img, (xshift, yshift), (0, 1))
    return img

In [ ]:
sample = train_X[2]
fig, axs = plt.subplots(1, 4, figsize=(15, 5))

axs[0].imshow(sample, cmap="gray")
axs[0].set_title("Original Image")

axs[1].imshow(shear(sample, 0.2), cmap="gray")
axs[1].set_title("Amount = 0.2")

axs[2].imshow(shear(sample, 0.4), cmap="gray")
axs[2].set_title("Amount = 0.4")

axs[3].imshow(shear(sample, 0.6), cmap="gray")
axs[3].set_title("Amount = 0.6")

plt.show()

Create an augmentation function which applies a random shear according to the constraint we provide:

In [ ]:
def augShear(sample, shearconstraint):
    """
    This function takes in a sample and a shear constraint and returns the augmented sample
    by shearing the sample by a random amount within the shear constraint

    sample: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    shearconstraint: the maximum shear by which the sample can be sheared

    returns: the augmented sample which is the input sample sheared by a random amount within the shear constraint
    """
    if shearconstraint == 0:
        return sample
    if len(sample.shape) == 2:
        # make sure the sample is 3 dimensional
        sample = np.expand_dims(sample, 0)
    amt = rng.random(len(sample))  # generate random numbers for shear
    amt = (amt - 0.5) * shearconstraint  # make the random shear constrained
    nsample = sample.copy()  # preallocate the augmented array to make it faster
    for ii in range(len(sample)):
        nsample[ii] = shear(sample[ii], amt[ii])
    return np.squeeze(nsample)  # take care if the input had only one sample.

Let us do a grid search to find the best shear constraint.

In [ ]:
shearconstraints = [
    0,
    0.2,
    0.4,
    0.6,
    0.8,
    1.0,
    1.2,
    1.4,
    1.6,
    1.8,
    2.0,
]  # the values we want to test
accuracies = np.zeros(
    len(shearconstraints), dtype=float
)  # we will save the values here

for ii in range(len(shearconstraints)):
    # create the augmented dataset
    augdata = train_X  # we include the original images also in the augmented dataset
    auglabel = train_y
    for jj in range(naugmentations):
        augdata = np.concatenate(
            (augdata, augShear(train_X, shearconstraints[ii]))
        )  # concatenate the augmented data to the set
        auglabel = np.concatenate(
            (auglabel, train_y)
        )  # the labels don't change when we augment

    # check the test accuracy
    testpred = NN(augdata, auglabel, test_X)
    accuracies[ii] = Accuracy(test_y, testpred)
    print(
        "Accuracy after shear augmentation constrained by",
        shearconstraints[ii],
        "is",
        accuracies[ii]*100,
        "%",
        flush=True,
    )

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
# plot the variation of accuracy
ax.plot(shearconstraints, accuracies)
ax.set_xlabel("angle")
ax.set_ylabel("accuracy")

# plot the maximum accuracy
maxind = np.argmax(accuracies)
plt.scatter(shearconstraints[maxind], accuracies[maxind], c="red")

### Augmentation 3: Rotation + Shear



We can do multiple augmentations at the same time. Here is a function to do both shear and rotation to the sample. In this case, we will have two hyperparameters.

In [ ]:
def augRotateShear(sample, angleconstraint, shearconstraint):
    """
    This function takes in a sample, an angle constraint and a shear constraint and returns the augmented sample
    by rotating the sample by a random angle within the angle constraint and shearing the sample by a random amount within the shear constraint

    sample: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    angleconstraint: the maximum angle by which the sample can be rotated
    shearconstraint: the maximum shear by which the sample can be sheared

    returns: the augmented sample which is the input sample rotated by a random angle within the angle constraint and sheared by a random amount within the shear constraint
    """
    if len(sample.shape) == 2:
        # make sure the sample is 3 dimensional
        sample = np.expand_dims(sample, 0)
    amt = rng.random(len(sample))  # generate random numbers for shear
    amt = (amt - 0.5) * shearconstraint  # make the random shear constrained
    angle = rng.random(len(sample))  # generate random numbers for angles
    # make the random angle constrained
    angle = (angle - 0.5) * angleconstraint
    nsample = sample.copy()  # preallocate the augmented array to make it faster
    for ii in range(len(sample)):
        nsample[ii] = rotate(
            shear(sample[ii], amt[ii]), angle[ii]
        )  # first apply shear, then rotate
    return np.squeeze(nsample)  # take care if the input had only one sample.

Since we have two hyperparameters, we have to do the grid search on a 2 dimensional matrix. We can use our previous experience to inform where to search for the best hyperparameters.

In [ ]:
shearconstraints = [
    0,
    0.2,
    0.4,
    0.6,
    0.8,
    1.0,
    1.2,
    1.4,
    1.6,
]  # the values we want to test
angleconstraints = [0, 10, 20, 30, 40, 50, 60]  # the values we want to test
# cartesian product of both
hyp = cartesian((shearconstraints, angleconstraints))

accuracies = np.zeros(len(hyp), dtype=float)  # we will save the values here

for ii in range(len(hyp)):
    # create the augmented dataset
    augdata = train_X  # we include the original images also in the augmented dataset
    auglabel = train_y
    for jj in range(naugmentations):
        augdata = np.concatenate(
            (augdata, augRotateShear(train_X, hyp[ii][0], hyp[ii][1]))
        )  # concatenate the augmented data to the set
        auglabel = np.concatenate(
            (auglabel, train_y)
        )  # the labels don't change when we augment

    # check the test accuracy
    testpred = NN(augdata, auglabel, test_X)
    accuracies[ii] = Accuracy(test_y, testpred)
    print(
        "Accuracy after augmentation shear:",
        hyp[ii][0],
        "angle:",
        hyp[ii][1],
        "is",
        accuracies[ii]*100,
        "%",
        flush=True,
    )

Accuracy after augmentation shear: 0.0 angle: 0.0 is 63.32 %
Accuracy after augmentation shear: 0.0 angle: 10.0 is 63.959999999999994 %
Accuracy after augmentation shear: 0.0 angle: 20.0 is 60.64000000000001 %
Accuracy after augmentation shear: 0.0 angle: 30.0 is 63.019999999999996 %
Accuracy after augmentation shear: 0.0 angle: 40.0 is 64.14999999999999 %
Accuracy after augmentation shear: 0.0 angle: 50.0 is 61.72 %
Accuracy after augmentation shear: 0.0 angle: 60.0 is 63.7 %
Accuracy after augmentation shear: 0.2 angle: 0.0 is 63.41 %
Accuracy after augmentation shear: 0.2 angle: 10.0 is 61.25000000000001 %
Accuracy after augmentation shear: 0.2 angle: 20.0 is 60.6 %
Accuracy after augmentation shear: 0.2 angle: 30.0 is 60.07 %
Accuracy after augmentation shear: 0.2 angle: 40.0 is 63.690000000000005 %
Accuracy after augmentation shear: 0.2 angle: 50.0 is 60.12 %
Accuracy after augmentation shear: 0.2 angle: 60.0 is 63.72 %
Accuracy after augmentation shear: 0.4 angle: 0.0 is 63.37000

Let us plot it two dimensionally to see which is the best value for the hyperparameters:

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
im = ax.imshow(
    accuracies.reshape((len(shearconstraints), len(angleconstraints))), cmap="hot"
)
ax.set_xlabel("Angle")
ax.set_ylabel("Shear")
ax.set_xticks(np.arange(len(angleconstraints)))
ax.set_xticklabels(angleconstraints)
ax.set_yticks(np.arange(len(shearconstraints)))
ax.set_yticklabels(shearconstraints)
plt.colorbar(im)

It seems that rotation and shear don't mix! The best accuracy is when rotation is zero.

## Questions
Try these questions for better understanding. You may not be able to solve all of them.
1. What is the best value for angle constraint and shear constraint you got? How much did the accuracy improve as compared to not using augmentations?
2. Can you increase the accuracy by increasing the number of augmentations from each sample?
3. Try implementing a few augmentations of your own and experimenting with them. A good reference is <a href=https://www.analyticsvidhya.com/blog/2019/12/image-augmentation-deep-learning-pytorch/>here. </a>
4. Try combining various augmentations. What is the highest accuracy you can get? What is the smallest training dataset you can take and still get accuracy above 50%?

Whenever you do any experiment, a good practice is to vary the hyperparameters gradually and create a graph of your results, like we did for gridsearch.

1AnsCould you clarify the specific problem or dataset you're referring to, as well as the model you're working with? This will help me give you more targeted insights regarding the best values for angle and shear constraints, and how augmentation affected your model's accuracy.

If you're referring to a general case in computer vision, here's a breakdown of how angle (rotation) and shear constraints in data augmentation can influence performance:

1. Angle Constraint (Rotation)

This refers to the degree of rotation applied to images during augmentation. Small rotations (typically between -15° to 15°) tend to work well for most tasks because they introduce variability without distorting the object in the image.

If the dataset contains objects that appear in various orientations, a wider range of angles might be appropriate (e.g., -30° to 30° or even -45° to 45°).

Effect on Accuracy: Rotation often improves accuracy by making the model more robust to different orientations. However, too much rotation can lead to distorted images and hurt performance. In general, modest improvements of 2-5% in validation/test accuracy are common with well-tuned rotations.


2. Shear Constraint

Shear refers to slanting the image, where one axis is stretched more than the other. Common values for shear range between 0.1 and 0.3 (or equivalently, 10° to 30° shear), depending on how much distortion your model can handle.

Shear augmentations are helpful when objects in the dataset are viewed from varying angles or perspectives.

Effect on Accuracy: Shear can improve performance by making the model invariant to perspective distortions, though it is less commonly used than rotation. It tends to contribute to small improvements (1-3%) in accuracy if the dataset benefits from perspective changes.


Comparison to No Augmentations

Typically, applying augmentations such as rotation and shear can lead to accuracy improvements of 5-10% depending on the dataset and model architecture. The improvement is more pronounced in small datasets or when the dataset has a lot of variability in object orientation or perspective.

Without augmentations, the model may overfit to the training data because it sees only a limited set of variations. With augmentations, the model becomes more robust and generalizes better to unseen data, leading to higher accuracy on the test set.


Key Considerations:

Dataset Type: The effectiveness of these augmentations depends heavily on the dataset. For example, in digit recognition tasks (like MNIST), small rotations and shear transformations often improve accuracy. In contrast, for tasks where object orientation is fixed (e.g., face recognition), too much rotation or shear may degrade performance.

Model Capacity: Augmentations tend to benefit deep models more, as they have the capacity to learn from the additional variability introduced by augmentations.


If you provide more details, I can give you a more specific recommendation.

2AnsYes, increasing the number of augmentations from each sample can improve accuracy, but the extent of improvement depends on several factors, such as the dataset size, task complexity, and model capacity. Here's how and why generating more augmentations from each sample can impact performance:

1. Increasing Dataset Variability

By generating multiple augmentations from each sample, you're artificially increasing the size of your dataset, which introduces more variability for the model to learn from. This helps prevent overfitting, especially when the original dataset is small or lacks diversity.

The model sees more diverse examples of the data, which can improve its ability to generalize to unseen samples.


2. Improved Generalization

With more augmentations, the model learns to be invariant to transformations such as rotations, flips, or color changes. This makes the model more robust to variations in real-world data, improving test accuracy.

For example, in tasks like image classification, augmenting each sample with multiple versions (e.g., rotated, sheared, cropped, etc.) can make the model less sensitive to irrelevant features like orientation or lighting.


3. Diminishing Returns

While more augmentations generally help, there are diminishing returns after a certain point. If too many augmentations are applied, the model might start to learn noise or unrealistic features that do not exist in real-world data.

At some point, the additional augmentations may no longer provide meaningful new information, and the computational cost increases without a proportional gain in accuracy.


4. Over-Augmentation Risks

If too many or overly aggressive augmentations are applied (e.g., excessive rotations, shear, or noise), the data might become distorted to the point where it no longer represents the true distribution. This can confuse the model and lead to worse performance.

It’s important to balance the number and type of augmentations so that they still reflect real-world variations.


5. Impact on Small vs. Large Datasets

Small Datasets: When the training data is limited, augmenting each sample multiple times can significantly improve model performance by acting as a form of regularization and reducing overfitting. The model becomes more exposed to variations, which it wouldn't have otherwise seen in a small dataset.

Large Datasets: For large datasets, the effect of augmenting each sample multiple times may be less pronounced since the dataset already contains substantial variability. In such cases, moderate augmentation may be enough to achieve good results, and further augmentations might not provide significant improvements.


Practical Considerations

1. Balanced Augmentations: Focus on realistic augmentations that represent the types of variability your model might encounter in the real world. For example, if you're building a model to recognize objects in photos, augmentations like random rotations, cropping, or lighting changes are appropriate. Overly extreme transformations may hurt performance.


2. Data Augmentation with Deep Learning: Deep models can handle a lot of data and benefit more from large amounts of augmentations, but shallow models may not leverage this increased variability as effectively. Make sure your model architecture is sufficiently complex to learn from the augmented data.


3. Use of Online Augmentation: Instead of generating and storing multiple augmentations per sample upfront (which could be memory-intensive), you can apply augmentations on-the-fly during training. This creates effectively infinite variations of the data without increasing storage requirements.


4. Fine-Tuning: You can first train your model without augmentations or with minimal augmentations, then progressively increase the number of augmentations in later stages of training. This can sometimes help stabilize learning.



Example of Augmentation Benefits

CIFAR-10 and ImageNet: In benchmark datasets like CIFAR-10 and ImageNet, data augmentation (especially random crops, horizontal flips, and color jittering) has been shown to significantly boost accuracy. Some models see an improvement of 2-5% just from augmenting each sample in different ways.


Conclusion:

Increasing the number of augmentations from each sample generally improves accuracy, particularly for small or medium-sized datasets, by introducing more data variability and improving generalization. However, the improvement has diminishing returns beyond a certain point, and overly aggressive augmentations can actually harm model performance. The key is to strike the right balance between the quantity and quality of augmentations, ensuring they reflect real-world variations.

3AnsIt seems you mentioned a reference, but I don’t have access to external resources directly. However, I can still help you implement common augmentations and suggest a process for experimenting with them.

Here’s a basic guide for implementing augmentations in popular machine learning libraries like TensorFlow/Keras or PyTorch, followed by some ideas on how to experiment with them.

Step 1: Implementing Common Augmentations

You can use a variety of augmentations such as rotations, flips, shear transformations, and brightness adjustments to make your dataset more diverse.

1. In TensorFlow/Keras:

Keras has a built-in ImageDataGenerator for augmentations, or you can use tf.image for more customized augmentations.

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator instance
datagen = ImageDataGenerator(
    rotation_range=30,         # Random rotations between -30 and 30 degrees
    width_shift_range=0.2,     # Random horizontal shifts
    height_shift_range=0.2,    # Random vertical shifts
    shear_range=0.2,           # Shear transformations
    zoom_range=0.2,            # Zooming in/out
    horizontal_flip=True,      # Random horizontal flips
    fill_mode='nearest'        # Filling empty space with nearest pixels
)

# Applying augmentations to training data
train_datagen = datagen.flow_from_directory('train_data_path', target_size=(150, 150), batch_size=32)

You can also use tf.image for custom augmentations:

import tensorflow as tf

def augment(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_rotation(image, angles=0.2)
    return image

# Apply augmentations to dataset
dataset = dataset.map(lambda x, y: (augment(x), y))

2. In PyTorch:

PyTorch provides torchvision.transforms for augmentations:

import torchvision.transforms as transforms

# Define augmentations
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),  # Random rotations
    transforms.RandomResizedCrop(224),  # Random cropping and resizing
    transforms.RandomHorizontalFlip(),  # Random horizontal flips
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Adjust brightness, contrast
    transforms.ToTensor(),  # Convert image to tensor
])

# Apply augmentations
train_dataset = torchvision.datasets.ImageFolder('train_data_path', transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

Step 2: Experimenting with Augmentations

To experiment with augmentations and observe their effects on accuracy, you can try varying the following:

1. Type of Augmentation:

Rotation: Experiment with rotation angles, from small rotations (±15°) to larger ones (±45°).

Flips: Use horizontal or vertical flips (depending on whether they make sense for your data).

Shear Transformations: Experiment with shearing in different directions and magnitudes.

Brightness, Contrast, and Saturation: Vary these to simulate different lighting conditions.


2. Number of Augmentations per Sample:

Generate multiple augmentations for each sample and compare performance. For example, for each image, you could apply 2–3 augmentations (rotation + flip + brightness adjustment) and then compare it to only applying one augmentation.


3. Intensity of Augmentations:

Try different ranges for augmentations (e.g., larger vs. smaller rotations or shears). This can help determine if the model performs better with subtle or more aggressive changes.


4. Without Augmentations vs. Augmentations:

Compare model performance with and without augmentations to quantify the effect of data augmentation. Start with no augmentations, then progressively add different augmentations to see how much the accuracy improves.


Step 3: Evaluating Performance

Track performance metrics such as accuracy and loss on the validation and test sets. You can log the performance after each experiment and visualize the improvements (or drop) in accuracy as you change the types and amounts of augmentations.

Example Process:

1. Baseline (No Augmentation):

Train the model without any augmentations to establish a baseline accuracy.



2. Adding One Augmentation at a Time:

Apply simple augmentations (e.g., random horizontal flips) and train the model again.

Observe how accuracy changes compared to the baseline.



3. Combining Multiple Augmentations:

Apply multiple augmentations (e.g., rotation + shear + brightness) and train.

Compare performance to both the baseline and the single-augmentation setup.



4. Vary Augmentation Intensity:

Increase or decrease the intensity of the augmentations (e.g., try smaller/larger rotations or contrast adjustments).

Record how this impacts validation and test accuracy.




If you have a specific reference or target model/dataset in mind, I can help you tailor the augmentation experiments further.

4AnsTo conduct the experiments of combining various augmentations and testing their impact on accuracy, as well as determining the smallest dataset that yields an accuracy above 50%, the following steps would be involved:

1. Combining Various Augmentations:

We’ll combine augmentations such as:

Rotation: Introduces random rotations to images.

Shear: Applies affine transformations to images.

Horizontal Flip: Randomly flips images horizontally.

Brightness, Contrast Adjustments: Simulate lighting conditions.

Zoom/Crop: Randomly zoom in or crop regions of the image.


For illustration purposes, let’s assume we're working with a dataset like CIFAR-10 (or any image classification dataset). You could run this experiment in TensorFlow/Keras or PyTorch.

Experiment Setup in TensorFlow/Keras:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Define augmentations
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train with data augmentation
batch_size = 64
epochs = 50  # Can be adjusted
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)
model.fit(train_generator, epochs=epochs, validation_data=(x_test, y_test))

This code will apply multiple augmentations and train a CNN on CIFAR-10. After training, you would get a test accuracy and could log it for comparison.

2. Testing on Smaller Training Dataset:

To determine the smallest dataset size that still achieves more than 50% accuracy, you can progressively reduce the size of the training dataset and retrain the model. Here's how to sample smaller datasets:

# Reduce the dataset size
small_train_size = 1000  # Starting with 1000 samples for training
x_small_train = x_train[:small_train_size]
y_small_train = y_train[:small_train_size]

# Train on smaller dataset with augmentations
train_generator_small = datagen.flow(x_small_train, y_small_train, batch_size=batch_size)
model.fit(train_generator_small, epochs=epochs, validation_data=(x_test, y_test))

You can gradually reduce the size of the training set (e.g., 500, 250, etc.) and retrain to see the point where the accuracy drops below 50%. Track how the augmentations compensate for the small dataset size.

Key Points:

1. Experiment with Data Size:

Start with 1000 training samples and progressively decrease the size. Use the augmentation pipeline to make the model more robust despite the small dataset size.

After each experiment, evaluate the model's accuracy and check whether it remains above 50%.



2. Combining Augmentations:

Experiment with different combinations of augmentations and measure their effect on validation accuracy.

Common combinations include rotation + flip + shear or zoom + crop + contrast.



3. Stopping Criteria:

When the model accuracy drops below 50%, note the corresponding dataset size.

You may find that with a well-chosen augmentation strategy, the model can perform surprisingly well on small datasets.




Expected Outcomes:

With Augmentations: You might achieve an accuracy of 50-70% even with a reduced dataset (e.g., 500-1000 samples) if the augmentations are varied and realistic.

Without Augmentations: On

> Exercise: Try to take 50 images of each digit and calculate the performance on test set.